In [6]:
import pandas as pd
import glob
import os
import re
from typing import Dict, List, Tuple

parent_dir = os.path.dirname(os.getcwd())
# date = "10.28"
date_in_filename = os.path.basename(parent_dir)
date=date_in_filename.split("_")[0]

extracted_code_dir = os.path.join(parent_dir, "extracted_code")

In [ ]:
# Find all extracted Python files
def get_extracted_python_files():
    """Get list of all Python files in the extracted_code directory"""
    if not os.path.exists(extracted_code_dir):
        print(f"❌ Extracted code directory not found: {extracted_code_dir}")
        return []
    
    python_files = glob.glob(os.path.join(extracted_code_dir, "*tpusg*.py"))
    python_files.sort()  # Sort for consistent processing order
    
    print(f"📁 Found {len(python_files)} Python files in {extracted_code_dir}")
    return python_files

# Get the list of files to execute
python_files = get_extracted_python_files()

# Show first few files as preview
if python_files:
    print("\n📋 First 5 Python files to execute:")
    for i, file_path in enumerate(python_files[:5]):
        filename = os.path.basename(file_path)
        print(f"  {i+1}. {filename}")
    
    if len(python_files) > 5:
        print(f"  ... and {len(python_files) - 5} more files")
else:
    print("❌ No Python files found to execute")
    print(f"Expected directory: {extracted_code_dir}")
    print("Make sure you've run the extraction process first.")

📁 Found 30 Python files in /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.10/extracted_code

📋 First 5 Python files to execute:
  1. 001_c26f_phi4_tpusg_0a9fc2a6.py
  2. 002_c26f_phi4_tpusg_c390bcf0.py
  3. 003_c26f_phi4_tpusg_c50e13ef.py
  4. 004_c26f_phi4_tpusg_e33f16c1.py
  5. 005_c26f_phi4_tpusg_7c98874d.py
  ... and 25 more files


In [8]:
import subprocess
import tempfile
import shutil
import uuid
import time
import threading
import queue
import logging
from datetime import datetime

class RemoteTPUExecutor:
    """Execute Python scripts on remote TPU device (Coral Dev Board)"""
    
    def __init__(self, remote_host='coral', remote_path='/home/mendel/tinyml_autopilot/tmp', 
                 remote_python='/home/mendel/tinyml_autopilot/tinyml-env/bin/python'):
        self.remote_host = remote_host
        self.remote_path = remote_path  
        self.remote_python = remote_python
        self.session_id = f"exec_{int(time.time())}"
        
        # Setup basic logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)
        
    def get_session_id(self):
        return f"[{self.session_id}] "
    
    def execute_code_remotely(self, code, script_name="unnamed_script"):
        """
        Execute code remotely via direct SSH file transfer and execution.
        
        Args:
            code (str): The Python code to execute remotely
            script_name (str): Name identifier for the script
            
        Returns:
            dict: Execution result with status, output, and error info
        """
        # Generate unique script name with timestamp
        script_id = f"script_{uuid.uuid4().hex[:8]}_{int(time.time())}"
        script_filename = f"{script_id}.py"
        remote_script_path = f"{self.remote_path}/{script_filename}"
        
        result = {
            'script_id': script_id,
            'script_name': script_name,
            'success': False,
            'output': '',
            'error': '',
            'execution_time': 0
        }
        
        start_time = time.time()
        
        try:
            # Check and kill any TPU device owner processes
            self._cleanup_tpu_device()
            
            # Create temporary local file with the code
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
                temp_file.write(code)
                local_script_path = temp_file.name
            
            # Save a copy for analysis
            analysis_dir = os.path.join("tmp")
            os.makedirs(analysis_dir, exist_ok=True)
            analysis_file_path = os.path.join(analysis_dir, f"{script_id}.py")
            shutil.copy2(local_script_path, analysis_file_path)
            
            # Commands for remote execution
            mkdir_command = ['ssh', self.remote_host, f'mkdir -p {self.remote_path}']
            scp_command = ['scp', local_script_path, f'{self.remote_host}:{remote_script_path}']
            ssh_command = ['ssh', self.remote_host, f'cd {self.remote_path} && {self.remote_python} {script_filename}']
            cleanup_command = ['ssh', self.remote_host, f'rm -f {remote_script_path}']
            kill_command = ['ssh', self.remote_host, f'pkill -f {script_id} || true']
            
            # Step 1: Create remote directory
            self.logger.info(self.get_session_id() + f"Creating remote directory: {self.remote_path}")
            mkdir_result = subprocess.run(mkdir_command, capture_output=True, text=True, timeout=30)
            
            if mkdir_result.returncode != 0:
                result['error'] = f"Failed to create remote directory: {mkdir_result.stderr}"
                return result
            
            # Step 2: Transfer script to remote machine
            self.logger.info(self.get_session_id() + f"Transferring script to remote: {remote_script_path}")
            scp_result = subprocess.run(scp_command, capture_output=True, text=True, timeout=60)
            
            if scp_result.returncode != 0:
                result['error'] = f"Failed to transfer script: {scp_result.stderr}"
                return result
            
            # Step 3: Execute script remotely
            self.logger.info(self.get_session_id() + f"Executing script remotely: {script_filename}")
            execution_error = self._stream_remote_execution(ssh_command, script_id)
            
            # Step 4: Cleanup
            self.logger.info(self.get_session_id() + f"Cleaning up remote script: {remote_script_path}")
            subprocess.run(kill_command, capture_output=True, text=True, timeout=30)
            cleanup_result = subprocess.run(cleanup_command, capture_output=True, text=True, timeout=30)
            
            if cleanup_result.returncode != 0:
                self.logger.warning(self.get_session_id() + f"Failed to cleanup remote file: {cleanup_result.stderr}")
            
            # Process results
            if execution_error is None:
                result['success'] = True
                result['output'] = "Execution completed successfully"
            else:
                result['error'] = execution_error
                
        except subprocess.TimeoutExpired as e:
            result['error'] = f"Remote execution timeout: {e}"
        except Exception as e:
            result['error'] = f"Remote execution failed: {e}"
        finally:
            # Cleanup local temporary file
            try:
                if 'local_script_path' in locals():
                    os.unlink(local_script_path)
            except Exception as cleanup_error:
                self.logger.warning(self.get_session_id() + f"Failed to cleanup local temp file: {cleanup_error}")
        
        result['execution_time'] = time.time() - start_time
        return result
    
    def _cleanup_tpu_device(self):
        """Kill any processes that might be using the TPU device"""
        try:
            check_result = subprocess.run(
                ['ssh', self.remote_host, "cat /sys/class/apex/apex_0/device_owner"], 
                capture_output=True, text=True, timeout=10
            )
            if check_result.returncode == 0:
                owner_pid = check_result.stdout.strip()
                if owner_pid and owner_pid != "0":
                    self.logger.warning(self.get_session_id() + f"TPU Device owned by PID {owner_pid}, killing process")
                    kill_cmd = ['ssh', self.remote_host, f'kill {owner_pid} || true']
                    subprocess.run(kill_cmd, capture_output=True, text=True, timeout=10)
        except:
            pass
    
    def _stream_remote_execution(self, ssh_command, script_id):
        """
        Execute SSH command with real-time output streaming.
        
        Args:
            ssh_command (list): SSH command to execute
            script_id (str): Unique script identifier for logging
            
        Returns:
            str: Error message if execution failed, None if successful
        """
        try:
            process = subprocess.Popen(
                ssh_command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=0,
                universal_newlines=True
            )
            
            self.logger.info(self.get_session_id() + f"🚀 Remote execution started for {script_id}")
            
            stdout_lines = []
            stderr_lines = []
            
            def read_stdout():
                try:
                    for line in iter(process.stdout.readline, ''):
                        if line:
                            line = line.rstrip('\n')
                            stdout_lines.append(line)
                    if stdout_lines:
                        if any('Error' in line for line in stdout_lines):
                            stderr_lines.extend(stdout_lines)
                        else:
                            self.logger.info(self.get_session_id() + f"[STDOUT] {stdout_lines}")
                except:
                    pass
            
            def read_stderr():
                try:
                    for line in iter(process.stderr.readline, ''):
                        if line:
                            line = line.rstrip('\n')
                            stderr_lines.append(line)
                    if stderr_lines and 'Traceback (most recent call last)' not in ' '.join(stderr_lines):
                        self.logger.error(self.get_session_id() + f"[STDERR] {stderr_lines}")
                except:
                    pass
            
            # Start reader threads
            stdout_thread = threading.Thread(target=read_stdout)
            stderr_thread = threading.Thread(target=read_stderr)
            stdout_thread.daemon = True
            stderr_thread.daemon = True
            
            stdout_thread.start()
            stderr_thread.start()
            
            # Wait for process completion with timeout
            timeout_seconds = 60
            start_time = time.time()
            
            while True:
                if process.poll() is not None:
                    break
                
                current_time = time.time()
                if current_time - start_time > timeout_seconds:
                    self.logger.error(self.get_session_id() + f"Remote execution timeout after {timeout_seconds} seconds")
                    process.terminate()
                    time.sleep(2)
                    if process.poll() is None:
                        process.kill()
                    return f"Remote execution timeout after {timeout_seconds} seconds"
                
                time.sleep(0.1)
            
            # Wait for threads to finish
            stdout_thread.join(timeout=5)
            stderr_thread.join(timeout=5)
            
            exit_code = process.returncode
            full_stdout = '\n'.join(stdout_lines)
            full_stderr = '\n'.join(stderr_lines)
            
            combined_output = ""
            if full_stdout:
                combined_output += full_stdout
            if full_stderr:
                if combined_output:
                    combined_output += "\n"
                combined_output += full_stderr
            
            if exit_code == 0:
                self.logger.info(self.get_session_id() + f"✅ Remote execution completed successfully for {script_id}")
                return None
            else:
                error_msg = f"❌ Remote execution failed with exit code {exit_code}"
                if full_stderr and 'Traceback' in full_stderr:
                    self.logger.error(self.get_session_id() + "Python traceback detected in remote execution:")
                    self.logger.error(self.get_session_id() + f"🔍 [REMOTE TRACEBACK] \n{full_stderr}")
                
                self.logger.error(self.get_session_id() + error_msg)
                return combined_output if combined_output else error_msg
                
        except Exception as e:
            error_msg = f"Failed to stream remote execution: {e}"
            self.logger.error(self.get_session_id() + error_msg)
            return error_msg

# Initialize the executor
executor = RemoteTPUExecutor()
print("🔧 Remote TPU Executor initialized")
print(f"   Remote Host: {executor.remote_host}")
print(f"   Remote Path: {executor.remote_path}")
print(f"   Remote Python: {executor.remote_python}")

🔧 Remote TPU Executor initialized
   Remote Host: coral
   Remote Path: /home/mendel/tinyml_autopilot/tmp
   Remote Python: /home/mendel/tinyml_autopilot/tinyml-env/bin/python


In [9]:
def execute_all_python_files(python_files, max_files=None, start_from=0):
    """
    Execute all Python files in the extracted_code directory one by one
    
    Args:
        python_files (list): List of Python file paths to execute
        max_files (int): Maximum number of files to execute (None for all)
        start_from (int): Index to start execution from
    
    Returns:
        dict: Summary of execution results
    """
    if not python_files:
        print("❌ No Python files to execute")
        return {}
    
    # Limit the number of files if specified
    files_to_execute = python_files[start_from:]
    if max_files:
        files_to_execute = files_to_execute[:max_files]
    
    print(f"🚀 Starting batch execution of {len(files_to_execute)} Python files")
    print(f"   Starting from index: {start_from}")
    if max_files:
        print(f"   Max files to execute: {max_files}")
    print(f"   Total files available: {len(python_files)}")
    print("=" * 60)
    
    execution_results = []
    successful_executions = 0
    failed_executions = 0
    
    for i, file_path in enumerate(files_to_execute):
        actual_index = start_from + i
        filename = os.path.basename(file_path)
        
        print(f"\n📄 [{actual_index + 1}/{len(python_files)}] Executing: {filename}")
        print("-" * 40)
        
        try:
            # Read the Python file content
            with open(file_path, 'r', encoding='utf-8') as f:
                code_content = f.read()
            
            # Extract metadata from the header comment
            metadata = extract_metadata_from_file(code_content)
            
            print(f"   📊 Metadata:")
            print(f"      Entry ID: {metadata.get('entry_id', 'Unknown')}")
            print(f"      Session: {metadata.get('session_id', 'Unknown')}")
            print(f"      Source: {metadata.get('source_file', 'Unknown')}")
            
            # Execute the code remotely
            print(f"   🚀 Starting remote execution...")
            result = executor.execute_code_remotely(code_content, filename)
            
            # Process results
            result['file_path'] = file_path
            result['filename'] = filename
            result['metadata'] = metadata
            result['index'] = actual_index
            
            if result['success']:
                print(f"   ✅ SUCCESS - Execution completed in {result['execution_time']:.2f}s")
                successful_executions += 1
            else:
                print(f"   ❌ FAILED - {result['error'][:100]}...")
                failed_executions += 1
            
            execution_results.append(result)
            
            # Brief pause between executions to avoid overwhelming the remote system
            time.sleep(2)
            
        except Exception as e:
            error_result = {
                'script_id': f"failed_{actual_index}",
                'script_name': filename,
                'file_path': file_path,
                'filename': filename,
                'success': False,
                'output': '',
                'error': f"Local execution setup failed: {e}",
                'execution_time': 0,
                'metadata': {},
                'index': actual_index
            }
            execution_results.append(error_result)
            failed_executions += 1
            print(f"   💥 SETUP FAILED - {e}")
    
    # Print summary
    print("\n" + "=" * 60)
    print("📊 BATCH EXECUTION SUMMARY")
    print("=" * 60)
    print(f"   Total files processed: {len(execution_results)}")
    print(f"   ✅ Successful executions: {successful_executions}")
    print(f"   ❌ Failed executions: {failed_executions}")
    print(f"   📈 Success rate: {successful_executions/len(execution_results):.1%}")
    
    return {
        'results': execution_results,
        'total_processed': len(execution_results),
        'successful': successful_executions,
        'failed': failed_executions,
        'success_rate': successful_executions/len(execution_results) if execution_results else 0
    }

def extract_metadata_from_file(code_content):
    """Extract metadata from the file header comment"""
    metadata = {}
    lines = code_content.split('\n')
    
    # Look for metadata in the header comment
    in_header = False
    for line in lines:
        line = line.strip()
        if line.startswith('"""'):
            in_header = not in_header
            continue
        
        if in_header and ':' in line:
            parts = line.split(':', 1)
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                metadata[key.lower().replace(' ', '_')] = value
    
    return metadata

def save_execution_results(batch_results, output_filename=None):
    """Save execution results to a CSV file for analysis"""
    if not batch_results or not batch_results['results']:
        print("❌ No results to save")
        return
    
    if output_filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f"execution_results_{timestamp}.csv"
    
    # Prepare data for CSV
    csv_data = []
    for result in batch_results['results']:
        metadata = result.get('metadata', {})
        csv_row = {
            'index': result.get('index', 0),
            'filename': result.get('filename', ''),
            'script_id': result.get('script_id', ''),
            'success': result.get('success', False),
            'execution_time': result.get('execution_time', 0),
            'error': result.get('error', '')[:500],  # Limit error message length
            'entry_id': metadata.get('entry_id', ''),
            'session_id': metadata.get('session_id', ''),
            'source_file': metadata.get('extracted_from', ''),
            'timestamp': metadata.get('timestamp', '')
        }
        csv_data.append(csv_row)
    
    # Save to CSV
    df = pd.DataFrame(csv_data)
    output_path = os.path.join(parent_dir, output_filename)
    df.to_csv(output_path, index=False)
    
    print(f"💾 Execution results saved to: {output_path}")
    print(f"   📊 {len(csv_data)} records saved")
    
    return output_path

print("🔧 Batch execution functions ready")
print("   Use execute_all_python_files() to start batch execution")
print("   Use save_execution_results() to save results to CSV")

🔧 Batch execution functions ready
   Use execute_all_python_files() to start batch execution
   Use save_execution_results() to save results to CSV


In [10]:
# Example usage and testing options

# Option 1: Test with just the first few files (recommended for initial testing)
def test_execution():
    """Test the execution system with just a few files"""
    if not python_files:
        print("❌ No Python files found. Run the file discovery cell first.")
        return
    
    print("🧪 Testing execution with first 3 files...")
    batch_results = execute_all_python_files(python_files, max_files=3, start_from=0)
    
    if batch_results:
        print("\n📊 Test Results:")
        for result in batch_results['results']:
            status = "✅ SUCCESS" if result['success'] else "❌ FAILED"
            print(f"   {result['filename']}: {status}")
        
        # Save test results
        save_execution_results(batch_results, "test_execution_results.csv")
    
    return batch_results

# Option 2: Execute all files (use with caution - could take a long time!)
def execute_all_files():
    """Execute ALL extracted Python files"""
    if not python_files:
        print("❌ No Python files found. Run the file discovery cell first.")
        return
    
    print(f"⚠️  WARNING: About to execute ALL {len(python_files)} Python files!")
    print("   This could take a very long time...")
    response = input("   Continue? (yes/no): ")
    
    if response.lower() != 'yes':
        print("❌ Execution cancelled")
        return
    
    print("🚀 Starting full batch execution...")
    batch_results = execute_all_python_files(python_files)
    
    if batch_results:
        # Save full results
        save_execution_results(batch_results, "full_execution_results.csv")
    
    return batch_results

# Option 3: Execute a specific range of files
def execute_range(start_index, count):
    """Execute a specific range of files"""
    if not python_files:
        print("❌ No Python files found. Run the file discovery cell first.")
        return
    
    if start_index >= len(python_files):
        print(f"❌ Start index {start_index} is beyond available files ({len(python_files)})")
        return
    
    print(f"🎯 Executing {count} files starting from index {start_index}")
    batch_results = execute_all_python_files(python_files, max_files=count, start_from=start_index)
    
    if batch_results:
        # Save range results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"execution_results_range_{start_index}_{start_index+count-1}_{timestamp}.csv"
        save_execution_results(batch_results, filename)
    
    return batch_results

# Option 4: Execute a single file by index
def execute_single_file(file_index):
    """Execute a single file by its index"""
    if not python_files:
        print("❌ No Python files found. Run the file discovery cell first.")
        return
    
    if file_index >= len(python_files):
        print(f"❌ File index {file_index} is beyond available files ({len(python_files)})")
        return
    
    filename = os.path.basename(python_files[file_index])
    print(f"🎯 Executing single file: {filename}")
    
    batch_results = execute_all_python_files(python_files, max_files=1, start_from=file_index)
    return batch_results

print("🎮 Execution Options Available:")
print("   test_execution()           - Test with first 3 files")
print("   execute_all_files()        - Execute ALL files (use with caution!)")
print("   execute_range(start, count) - Execute specific range of files")
print("   execute_single_file(index) - Execute a single file by index")
print("")
print("💡 Recommended: Start with test_execution() to verify the system works")
print("   Then use execute_range() for controlled batch processing")

# Show available files again for reference
if python_files:
    print(f"\n📋 Available files (total: {len(python_files)}):")
    for i, file_path in enumerate(python_files[:10]):  # Show first 10
        filename = os.path.basename(file_path)
        print(f"   {i:2d}. {filename}")
    if len(python_files) > 10:
        print(f"   ... and {len(python_files) - 10} more files")
    
    print(f"\n🚀 Ready to execute! Try: test_execution()")
else:
    print(f"\n❌ No files found in {extracted_code_dir}")
    print("   Make sure you've run the extraction process first")

🎮 Execution Options Available:
   test_execution()           - Test with first 3 files
   execute_all_files()        - Execute ALL files (use with caution!)
   execute_range(start, count) - Execute specific range of files
   execute_single_file(index) - Execute a single file by index

💡 Recommended: Start with test_execution() to verify the system works
   Then use execute_range() for controlled batch processing

📋 Available files (total: 30):
    0. 001_c26f_phi4_tpusg_0a9fc2a6.py
    1. 002_c26f_phi4_tpusg_c390bcf0.py
    2. 003_c26f_phi4_tpusg_c50e13ef.py
    3. 004_c26f_phi4_tpusg_e33f16c1.py
    4. 005_c26f_phi4_tpusg_7c98874d.py
    5. 006_c26f_phi4_tpusg_1216c207.py
    6. 007_c26f_phi4_tpusg_d6b8dbd1.py
    7. 008_c26f_phi4_tpusg_e1ebb966.py
    8. 009_c26f_phi4_tpusg_08184e66.py
    9. 010_c26f_phi4_tpusg_a472014d.py
   ... and 20 more files

🚀 Ready to execute! Try: test_execution()


In [12]:
execute_all_files()

⚠️  WARNING: About to execute ALL 30 Python files!
   This could take a very long time...
🚀 Starting full batch execution...
🚀 Starting batch execution of 30 Python files
   Starting from index: 0
   Total files available: 30

📄 [1/30] Executing: 001_c26f_phi4_tpusg_0a9fc2a6.py
----------------------------------------
   📊 Metadata:
      Entry ID: 0a9fc2a6
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...
🚀 Starting full batch execution...
🚀 Starting batch execution of 30 Python files
   Starting from index: 0
   Total files available: 30

📄 [1/30] Executing: 001_c26f_phi4_tpusg_0a9fc2a6.py
----------------------------------------
   📊 Metadata:
      Entry ID: 0a9fc2a6
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...


2025-08-12 07:05:49,733 - INFO - [exec_1754971292] Creating remote directory: /home/mendel/tinyml_autopilot/tmp
2025-08-12 07:05:50,421 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_8a893b5d_1754971549.py
2025-08-12 07:05:50,421 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_8a893b5d_1754971549.py
2025-08-12 07:05:51,201 - INFO - [exec_1754971292] Executing script remotely: script_8a893b5d_1754971549.py
2025-08-12 07:05:51,206 - INFO - [exec_1754971292] 🚀 Remote execution started for script_8a893b5d_1754971549
2025-08-12 07:05:51,201 - INFO - [exec_1754971292] Executing script remotely: script_8a893b5d_1754971549.py
2025-08-12 07:05:51,206 - INFO - [exec_1754971292] 🚀 Remote execution started for script_8a893b5d_1754971549
2025-08-12 07:06:40,846 - INFO - [exec_1754971292] ✅ Remote execution completed successfully for script_8a893b5d_1754971549
2025-08-12 07:06:40,849 - INFO - [exec

   ✅ SUCCESS - Execution completed in 53.31s

📄 [2/30] Executing: 002_c26f_phi4_tpusg_c390bcf0.py
----------------------------------------
   📊 Metadata:
      Entry ID: c390bcf0
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...

📄 [2/30] Executing: 002_c26f_phi4_tpusg_c390bcf0.py
----------------------------------------
   📊 Metadata:
      Entry ID: c390bcf0
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...


2025-08-12 07:06:45,056 - INFO - [exec_1754971292] Creating remote directory: /home/mendel/tinyml_autopilot/tmp
2025-08-12 07:06:45,867 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_82307928_1754971604.py
2025-08-12 07:06:45,867 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_82307928_1754971604.py
2025-08-12 07:06:46,721 - INFO - [exec_1754971292] Executing script remotely: script_82307928_1754971604.py
2025-08-12 07:06:46,727 - INFO - [exec_1754971292] 🚀 Remote execution started for script_82307928_1754971604
2025-08-12 07:06:46,721 - INFO - [exec_1754971292] Executing script remotely: script_82307928_1754971604.py
2025-08-12 07:06:46,727 - INFO - [exec_1754971292] 🚀 Remote execution started for script_82307928_1754971604
2025-08-12 07:07:36,279 - INFO - [exec_1754971292] ✅ Remote execution completed successfully for script_82307928_1754971604
2025-08-12 07:07:36,282 - INFO - [exec

   ✅ SUCCESS - Execution completed in 53.44s

📄 [3/30] Executing: 003_c26f_phi4_tpusg_c50e13ef.py
----------------------------------------
   📊 Metadata:
      Entry ID: c50e13ef
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...

📄 [3/30] Executing: 003_c26f_phi4_tpusg_c50e13ef.py
----------------------------------------
   📊 Metadata:
      Entry ID: c50e13ef
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...


...

2025-08-12 07:31:41,532 - INFO - [exec_1754971292] Creating remote directory: /home/mendel/tinyml_autopilot/tmp
2025-08-12 07:31:42,302 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_7782419b_1754973100.py
2025-08-12 07:31:42,302 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_7782419b_1754973100.py
2025-08-12 07:31:43,112 - INFO - [exec_1754971292] Executing script remotely: script_7782419b_1754973100.py
2025-08-12 07:31:43,118 - INFO - [exec_1754971292] 🚀 Remote execution started for script_7782419b_1754973100
2025-08-12 07:31:43,112 - INFO - [exec_1754971292] Executing script remotely: script_7782419b_1754973100.py
2025-08-12 07:31:43,118 - INFO - [exec_1754971292] 🚀 Remote execution started for script_7782419b_1754973100
2025-08-12 07:32:32,685 - INFO - [exec_1754971292] ✅ Remote execution completed successfully for script_7782419b_1754973100
2025-08-12 07:32:32,693 - INFO - [exec

   ✅ SUCCESS - Execution completed in 53.46s

📄 [30/30] Executing: 030_c26f_phi4_tpusg_b0500f27.py
----------------------------------------
   📊 Metadata:
      Entry ID: b0500f27
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...

📄 [30/30] Executing: 030_c26f_phi4_tpusg_b0500f27.py
----------------------------------------
   📊 Metadata:
      Entry ID: b0500f27
      Session: phi4_c26f_tpusg_batch
      Source: Unknown
   🚀 Starting remote execution...


2025-08-12 07:32:37,099 - INFO - [exec_1754971292] Creating remote directory: /home/mendel/tinyml_autopilot/tmp
2025-08-12 07:32:38,082 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_b9a2b00f_1754973156.py
2025-08-12 07:32:38,082 - INFO - [exec_1754971292] Transferring script to remote: /home/mendel/tinyml_autopilot/tmp/script_b9a2b00f_1754973156.py
2025-08-12 07:32:39,600 - INFO - [exec_1754971292] Executing script remotely: script_b9a2b00f_1754973156.py
2025-08-12 07:32:39,600 - INFO - [exec_1754971292] Executing script remotely: script_b9a2b00f_1754973156.py
2025-08-12 07:32:39,660 - INFO - [exec_1754971292] 🚀 Remote execution started for script_b9a2b00f_1754973156
2025-08-12 07:32:39,660 - INFO - [exec_1754971292] 🚀 Remote execution started for script_b9a2b00f_1754973156
2025-08-12 07:33:29,519 - INFO - [exec_1754971292] ✅ Remote execution completed successfully for script_b9a2b00f_1754973156
2025-08-12 07:33:29,522 - INFO - [exec

   ✅ SUCCESS - Execution completed in 54.79s

📊 BATCH EXECUTION SUMMARY
   Total files processed: 30
   ✅ Successful executions: 30
   ❌ Failed executions: 0
   📈 Success rate: 100.0%
💾 Execution results saved to: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.10/full_execution_results.csv
   📊 30 records saved

📊 BATCH EXECUTION SUMMARY
   Total files processed: 30
   ✅ Successful executions: 30
   ❌ Failed executions: 0
   📈 Success rate: 100.0%
💾 Execution results saved to: /Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.10/full_execution_results.csv
   📊 30 records saved


{'results': [{'script_id': 'script_8a893b5d_1754971549',
   'script_name': '001_c26f_phi4_tpusg_0a9fc2a6.py',
   'success': True,
   'output': 'Execution completed successfully',
   'error': '',
   'execution_time': 53.30786681175232,
   'file_path': '/Users/hann/Projects/reference-benchmark-tinyml_llm/data_analysis/2025/08.10/extracted_code/001_c26f_phi4_tpusg_0a9fc2a6.py',
   'filename': '001_c26f_phi4_tpusg_0a9fc2a6.py',
   'metadata': {'extracted_from': 'raw_phi4_c26f_tpusg_batch',
    'entry_id': '0a9fc2a6',
    'entry_name': '0a9f_tpu_sketch_generator',
    'session_id': 'phi4_c26f_tpusg_batch',
    'timestamp': '2025-08-10T11:37:14.278000+00:00',
    'tags': 'benchmark, phi4:latest, tpu_sketch_generator'},
   'index': 0},
  {'script_id': 'script_82307928_1754971604',
   'script_name': '002_c26f_phi4_tpusg_c390bcf0.py',
   'success': True,
   'output': 'Execution completed successfully',
   'error': '',
   'execution_time': 53.437453746795654,
   'file_path': '/Users/hann/Project